In [1]:
import pandas as pd
import numpy as np
import os 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, classification_report, precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

In [2]:
#Chargement du DF
data = pd.read_excel('Copie de Coeur.xlsx')

In [3]:
#Travailler avec une copie du data Frame pour éviter de le rechager en cas d'êrreur
df = data.copy()

In [4]:
#Afficher les informations relatives au DF
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGE          918 non-null    int64  
 1   SEXE         918 non-null    object 
 2   TDT          918 non-null    object 
 3   PAR          918 non-null    int64  
 4   CHOLESTEROL  918 non-null    int64  
 5   GAJ          918 non-null    int64  
 6   ECG          918 non-null    object 
 7   FCMAX        918 non-null    int64  
 8   ANGINE       918 non-null    object 
 9   DEPRESSION   918 non-null    float64
 10  PENTE        918 non-null    object 
 11  CŒUR         918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [38]:
#Normalisation des variables quantitatives
for col in df.drop('CŒUR', axis =1).select_dtypes(np.number).columns:
    df[col] = (df[col] - df[col].mean())/ df[col].std()


In [39]:
#Vérification des résultats
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,-1.432359,0.515671,-1.283640,0.410685,0.824621,-0.551041,0.017245,1.382175,-0.823108,-0.831979,-1.111948,0
1,-0.478223,-1.937107,1.669697,1.490940,-0.171867,-0.551041,0.017245,0.753746,-0.823108,0.105606,0.964895,1
2,-1.750404,0.515671,-1.283640,-0.129442,0.769768,-0.551041,1.600347,-1.524307,-0.823108,-0.831979,-1.111948,0
3,-0.584238,-1.937107,-0.299194,0.302660,0.138964,-0.551041,0.017245,-1.131539,1.213585,0.574398,0.964895,1
4,0.051853,0.515671,1.669697,0.950812,-0.034736,-0.551041,0.017245,-0.581664,-0.823108,-0.831979,-1.111948,0


In [40]:
#Encodage des variables qualitatives
for col in df.select_dtypes('object').columns:
    df[col] = df[col].astype('category').cat.codes

In [41]:
#Vérification des résultats
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,-1.432359,0.515671,-1.283640,0.410685,0.824621,-0.551041,0.017245,1.382175,-0.823108,-0.831979,-1.111948,0
1,-0.478223,-1.937107,1.669697,1.490940,-0.171867,-0.551041,0.017245,0.753746,-0.823108,0.105606,0.964895,1
2,-1.750404,0.515671,-1.283640,-0.129442,0.769768,-0.551041,1.600347,-1.524307,-0.823108,-0.831979,-1.111948,0
3,-0.584238,-1.937107,-0.299194,0.302660,0.138964,-0.551041,0.017245,-1.131539,1.213585,0.574398,0.964895,1
4,0.051853,0.515671,1.669697,0.950812,-0.034736,-0.551041,0.017245,-0.581664,-0.823108,-0.831979,-1.111948,0


In [42]:
#Séparer la variable cible (coeur) et les variables explicatives
y = df['CŒUR']
x = df.drop('CŒUR', axis = 1)

In [43]:
#Subdivision du jeu de données en apprentissage et en test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =1)
#train_set, test_set = train_test_split(df,test_size =0.2, random_state =1)

In [44]:
#Vérifier le résultat
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 11) (184, 11) (734,) (184,)


In [45]:
#Création d'un objet lr de la classe LogisticRegression
lr = LogisticRegression(solver ='newton-cg', random_state =1)

In [71]:
#Apprentissage du modèle
model_lr = lr.fit(x_train, y_train)
#model = lr.fit(train_set.drop('CŒUR', axis = 1), train_set['CŒUR'])

In [72]:
#Probabilité d'appartenance à l'une des classes
predict_proba_lr = model.predict_proba(x_test)


In [73]:
#Application du modèle au données de test
y_pred_lr = model.predict(x_test)
y_pred_lr[:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [74]:
#matrice de confusion
mc_lr = confusion_matrix(y_test,y_pred_lr)
mc_lr

array([[ 66,   8],
       [  8, 102]], dtype=int64)

In [75]:
#Taux de bonnes prédictions
accuracy_lr = accuracy_score(y_test, y_pred_lr)
accuracy_lr
#Conclusion : le modèle produit 92 % de bonne prédiction.

0.9130434782608695

In [76]:
#Sensibilité
recall_lr = recall_score(y_test, y_pred_lr)
recall_lr
#conclusion : lorsqu'un individu est malade, nous avons 94 % de chance que le modèle le prédise.

0.9272727272727272

In [77]:
#Précision
precision_lr = precision_score(y_test, y_pred_lr)
precision_lr
#Conclusion : lorsque le modèle prédit qu'un individu est malade, nous avons 92 % de chace que cela soit vraie.

0.9272727272727272

In [78]:
#plus de détails avec classification report
cr = classification_report(y_test, y_pred_lr)

In [79]:
#Modèle Trival
df['CŒUR'].value_counts()/df.shape[0]
#Conclusion : si nous nous basons sur le hasard pour faire nos prédiction, nous aurons 55.37 % de bonne prédiction.

1    0.553377
0    0.446623
Name: CŒUR, dtype: float64

In [80]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
model.score(x_train, y_train)

0.8474114441416893

In [81]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
model.score(x_test, y_test)

0.9130434782608695

# DecisionTree

In [82]:
#Création d'un objet model de la classe DecisionTreeClassifier
model_AD=DecisionTreeClassifier(criterion='gini')
#Gini est une formule mathematique de la fonction CART

In [83]:
#Apprentissage du modèle
model_AD.fit(x_train,y_train)

DecisionTreeClassifier()

In [84]:
#Subdivision du jeu de données en apprentissage et en test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =1)
#train_set, test_set = train_test_split(df,test_size =0.2, random_state =1)

In [85]:
#Vérifier le résultat
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 11) (184, 11) (734,) (184,)


In [86]:
#Probabilité d'appartenance à l'une des classes
predict_proba_AD = model_AD.predict_proba(x_test)


In [87]:
#Application du modèle au données de test
y_pred_AD = model_AD.predict(x_test)


In [88]:
#matrice de confusion
mc_AD = confusion_matrix(y_test, y_pred_AD)


In [89]:
#Taux de bonnes prédictions
accuracy_AD = accuracy_score(y_test, y_pred_AD)
accuracy_AD
#Conclusion : le modèle produit77 % de bonne prédiction.

0.7771739130434783

In [90]:
#Sensibilité
recall_AD = recall_score(y_test, y_pred_AD)
recall_AD
#conclusion : lorsqu'un individu est malade, nous avons 73 % de chance que le modèle le prédise.

0.7181818181818181

In [91]:
#Précision
precision_AD = precision_score(y_test, y_pred_AD)
precision_AD
#Conclusion : lorsque le modèle prédit qu'un individu est malade, nous avons 87,9 % de chace que cela soit vraie.

0.8876404494382022

In [92]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
model_AD.score(x_test, y_test)

0.7771739130434783

In [93]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
model_AD.score(x_train, y_train)

1.0

# Comparaison des scores entre la Régression Linéaire et L'Arbre de Décision

In [94]:
#visualisation de la performance des deux models

In [95]:
#1.regressionlogistic
print("Ceci est le score de la REgression sur les données Tests \n",model_lr.score(x_test,y_test))

#1.DecisionTree
print("Ceci est le score de l'Arbre de Décision sur les données Tests \n",model_AD.score(x_test,y_test))

Ceci est le score de la REgression sur les données Tests 
 0.9130434782608695
Ceci est le score de l'Arbre de Décision sur les données Tests 
 0.7771739130434783


In [96]:
#matrice de confusion de decisionTree
print("matrice de confusion de decisionTree : \n ",mc_lr)

#matrice de confusion de decisionTree
print("matrice de confusion de decisionTree : \n ",mc_AD)

matrice de confusion de decisionTree : 
  [[ 66   8]
 [  8 102]]
matrice de confusion de decisionTree : 
  [[64 10]
 [31 79]]


In [97]:
#Comparaison des sensibilités
   

In [98]:
 # Sensibité régression logistique
print("Sensibité régression logistique : \n ",recall_lr)

    # Sensibilité arbre de décision
print("Sensibilité arbre de décision : \n ",recall_AD)

Sensibité régression logistique : 
  0.9272727272727272
Sensibilité arbre de décision : 
  0.7181818181818181


In [99]:
#Précision


In [100]:
# Précision régression logistique
print("Précision régression logistique : \n",precision_lr)

# Précision arbre de décision
print("Précision arbre de décision : \n",precision_AD)

Précision régression logistique : 
 0.9272727272727272
Précision arbre de décision : 
 0.8876404494382022


In [101]:
#Comparaison au niveau des  Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
if model.score(x_train, y_train) > model_AD.score(x_train, y_train):
    print("Sur l'entrainement, le resultat de la Régression Logistique est supérieure à celui de l'Arbre de Décision")
else:
    print("Sur l'entrainement, le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision")

Sur l'entrainement, le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision


In [102]:
#Comparaison au niveau des Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
if model.score(x_test, y_test) > model_AD.score(x_test, y_test):
    print("Sur l'entrainement,le resultat de la Régression Logistique est supérieure à celui de l'Arbre de Décision")
else:
    print("Sur l'entrainement,le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision")

Sur l'entrainement,le resultat de la Régression Logistique est supérieure à celui de l'Arbre de Décision
